In [1]:
import os
import sys
import tensorflow as tf
import numpy      as np

In [2]:
InputFile      = '../input/O3_UMN/'


WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
SurQCTFldr     = WORKSPACE_PATH + '/SurQCT/surqct/'

print("\n======================================================================================================================================")
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))


 TensorFlow version: 2.4.1
 Eager execution: True


In [3]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
# from Reading import read_data, read_losseshistory
sys.path.insert(0, SurQCTFldr  + '/src/Plotting/')
from Plotting import plot_losseshistory
# sys.path.insert(0, SurQCTFldr  + '/src/Saving/')
# from Saving import save_parameters, save_data


InputFile = '../input/O3_UMN/KExcit/'
print("[SurQCT]:   Calling SurQCT with Input File = ", InputFile)
sys.path.insert(0, InputFile)
#===================================================================================================================================

#===================================================================================================================================
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

PathToFldr = InputData.PathToRunFld
try:
    os.makedirs(PathToFldr)
except OSError as e:
    pass
    
PathToFldr = InputData.PathToFigFld
try:
    os.makedirs(PathToFldr)
except OSError as e:
    pass

#===================================================================================================================================


#===================================================================================================================================
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model
# if (InputData.ApproxModel == 'FNN'):
#     from Model import FNN

sys.path.insert(0, SurQCTFldr  + '/src/RatesType/' + InputData.RatesType + '/')
from RatesType import generate_trainingdata, plot_prediction, generate_predictiondata

#===================================================================================================================================


[SurQCT]: Loading Modules and Functions ...
[SurQCT]:   Calling SurQCT with Input File =  ../input/O3_UMN/KExcit/

[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...

[SurQCT]: Loading Final Modules ... 


In [4]:
#===================================================================================================================================
print("\n[SurQCT]: Generating Data ... ")

InputData, TrainData, ValidData, AllData, TestData, ExtraData = generate_trainingdata(InputData)

#===================================================================================================================================


[SurQCT]: Generating Data ... 
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2.csv
[SurQCT]:   Generating Training and Validation Data Points
[SurQCT]:     Translational Temperature: T = 10000K
[SurQCT]:       Reading HDF5 File from: /home/venturi/WORKSPACE//Air_Database/HDF5_Database_HighAccuracy/O3_UMN.hdf5 for Excitation Rates at Temperature 10000K
[SurQCT]:       Reading Sampled Initial Levels List from File: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2_Sampled_10000K.csv
[SurQCT]:       Vector of Initial Levels: iIdxVec =  [  45   93  151  219  292  372  459  551  647  749  857 1276  968 1165
 1231  821 1823 1427  712  818 1339  938 1145 1294 1550 2098 2447 3322
 3784 2879 5276 5462 5332 3400 3643 4613 3904 4706 5362 5765 5980 4683
 4477 4509 4582]
[SurQCT]:       Now the Data Matrix contains  87951  Data Points
[SurQCT]:   Generating Labeled Test Data Points
[SurQCT]:       Reading HDF5 File from:

In [5]:
#===================================================================================================================================
print('\n[SurQCT]: Initializing ML Model ... ')

NN = model(InputData, TrainData, ValidData)

#===================================================================================================================================


[SurQCT]: Initializing ML Model ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'rMin_i', 'rMax_i', 'VMin_i', 'VMax_i', 'Tau_i', 'ri_i', 'ro_i', 'TTran_i', 'EVib_Delta', 'ERot_Delta', 'rMin_Delta', 'rMax_Delta', 'VMin_Delta', 'VMax_Delta', 'Tau_Delta', 'ri_Delta', 'ro_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Defining ML Model from Scratch
[SurQCT]:   Compiling ML Model with Loss and Optimizer
[SurQCT]:   Saving ML Model in File: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/KInel_Test1/MyModel
INFO:tensorflow:Assets written to: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/KInel_Test1/MyModel/assets
[SurQCT]:   Summarizing ML Model Structure:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0           

In [6]:
#===================================================================================================================================
if (InputData.TrainIntFlg > 0):


    if (InputData.TrainIntFlg == 1):
        print('\n[SurQCT]: Reading the ML Model Parameters ... ')

        NN.load_params(InputData)


    print('\n[SurQCT]: Training the ML Model ... ')

    History = NN.train(InputData)


    print('\n[SurQCT]: Plotting the Losses Evolution ... ')

    plot_losseshistory(InputData, History)


else:

    print('\n[SurQCT]: Reading the ML Model Parameters ... ')

    NN.load_params(InputData)

#===================================================================================================================================


[SurQCT]: Reading the ML Model Parameters ... 
[SurQCT]:   Loading ML Model Parameters from File:  /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/KInel_Test1/Params//ModelCheckpoint/cp-2574.ckpt


In [20]:
#===================================================================================================================================

if (InputData.PlotIntFlg >= 1):

    print('\n[SurQCT]: Evaluating the ML Model at the Training Data and Plotting the Results ... ')

    xAll      = AllData[0]
    yAll      = AllData[1]
    yPred     = NN.Model.predict(xAll[NN.xTrainingVar])

    plot_prediction(InputData, 'Train', InputData.TTranVecTrain, xAll, yAll, yPred)

#===================================================================================================================================



#===================================================================================================================================

if (InputData.TestIntFlg >= 1):

    if (InputData.PlotIntFlg >= 1):

        print('\n[SurQCT]: Evaluating the ML Model at the Test Data and Plotting the Results ... ')

        xTest     = TestData[0]
        yTest     = TestData[1]
        yPred     = NN.Model.predict(xTest[NN.xTrainingVar])

        plot_prediction(InputData, 'Test', InputData.TTranVecTest, xTest, yTest, yPred)

#===================================================================================================================================



#===================================================================================================================================

if (InputData.TestIntFlg >= 1):

    if (InputData.PlotIntFlg >= 1):

        print('\n[SurQCT]: Evaluating the ML Model at the Test Data and Plotting the Results ... ')

        xExtra    = ExtraData
        yPred     = NN.Model.predict(xExtra[NN.xTrainingVar])
        yData     = []

        plot_prediction(InputData, 'Extra', InputData.TTranVecExtra, xExtra, yData, yPred)

#===================================================================================================================================

1000

In [7]:
#===================================================================================================================================

if (InputData.PredictIntFlg >= 1):

    print('\n[SurQCT]: Generating Rate Matrixes ... ')

    TTran = 10000.0
    KExcitMat = generate_predictiondata(InputData, NN, TTran)
    print(KExcitMat)
    print(KExcitMat[9,:])

#===================================================================================================================================

[SurQCT]:   Generating Rate Matrix:   1%|          | 1/99 [00:00<00:15,  6.38it/s]


[SurQCT]: Generating Rate Matrixes ... 
[SurQCT]:   Generating Rate Matrix at Translational Temperature: T = 10000K
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2.csv


[SurQCT]:   Generating Rate Matrix: 100%|██████████| 99/99 [00:05<00:00, 17.66it/s]

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.81027927e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.81600219e-12 2.39212782e-11 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[1.24223266e-12 6.18836146e-12 1.22749805e-11 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [16]:
KExcitMat[99,6]

8.656217290268359e-14